## Data

### Done

1. load using pandas

## State Representation

### Done

1. Transform categorical idices
2. normalise cost

### TODO

1. continuous features bucketised ?
2. weights (real values floating point numbers, normalised (?))



In [39]:
import random
import pandas as pd
pd.options.display.max_columns = None
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [40]:
raw_data = pd.read_csv('./src/data/insurance.csv')
raw_features = pd.DataFrame(raw_data, columns=raw_data.columns.drop('charges'))
raw_labels = pd.DataFrame(raw_data.charges)

In [41]:
# we use this to map the categorical features into indices, we can map back later for presentation
def extract_category_mappings(dataframe, category_columns):
    to_map = lambda uniques: {v:k[0] for k, v in np.ndenumerate(uniques)}
    return [(to_map(dataframe[col].unique()), col) for col in category_columns]

In [42]:
mappings = extract_category_mappings(raw_features, ['sex', 'smoker', 'region'])

In [43]:
# transform the categorical features to binary representations of their indices;
# we might want to bucketize the continuous features?
def transform_features(dataframe, mappings):
    transformed_df = pd.DataFrame(dataframe, columns=['age', 'bmi', 'children'])
    
    for mapping, name in mappings:
        for i in mapping.values():
            transformed_df[name + str(i)] = dataframe[name].apply(lambda x: int(mapping[x] == i))

    return transformed_df

In [44]:
features = transform_features(raw_features, mappings)

In [45]:
scaler = preprocessing.MinMaxScaler()

def normalize(series, columns, scaler):
    return scaler.fit_transform(series[columns])

def denormalize(series, columns, scaler):
    return scaler.inverse_transform(series[columns])

labels = raw_labels
labels['charges'] = normalize(raw_labels, ['charges'], scaler)
print(labels.head())


    charges
0  0.251611
1  0.009636
2  0.053115
3  0.333010
4  0.043816


In [46]:
X_train, X_test, y_train, y_test = train_test_split(features, labels)

In [47]:
print(X_train.head())
print(X_test.head())
print(y_train.head())
print(y_test.head())

      age     bmi  children  sex0  sex1  smoker0  smoker1  region0  region1  \
375    23  28.310         0     1     0        1        0        0        0   
875    23  28.120         0     1     0        0        1        0        0   
1043   28  25.800         0     1     0        0        1        1        0   
987    45  27.645         1     1     0        0        1        0        0   
61     25  33.660         4     0     1        0        1        0        1   

      region2  region3  
375         1        0  
875         1        0  
1043        0        0  
987         1        0  
61          0        0  
      age     bmi  children  sex0  sex1  smoker0  smoker1  region0  region1  \
394    46  32.300         2     1     0        0        1        0        0   
115    60  28.595         0     0     1        0        1        0        0   
1156   19  44.880         0     0     1        1        0        0        1   
106    19  28.400         1     1     0        0        1  

In [48]:
#TODO warning, actually setting values on y_train if we run this
#test_denormalized_labels = y_train
#test_denormalized_labels['charges'] = denormalize(y_train, ['charges'], scaler)

#print(test_denormalized_labels.head())



# we probably only actually care about denormalising on predictions

In [49]:
X_train.size

11033

In [116]:
class GeneticAlgorithm:
    def __init__(self, parameters, features, labels):
        self.parameters = parameters
        # TODO from params
        self.population_size = 10
        self.features = features
        self.labels = labels
        self.population = [self.initial_weights() for i in range(self.population_size)]
        
    def initial_weights(self):
        shape_rows, shape_cols = self.features.shape
        random_chromosome = lambda cols: [random.random() for c in range(cols)]
        return pd.DataFrame([random_chromosome(shape_cols) for r in range(shape_rows)], columns=self.features.columns, index=self.features.index)

    def next_generation(self):
        # move from current generation
        pass
    
    def fitness(self, chromosome):
        #foo = self.features.apply(lambda f: np.dot(f, chromosome))
        [print(type(i)) for i in chromosome.iloc[375]]
        #print(self.features.dot(chromosome))
    

In [117]:
g = GeneticAlgorithm({}, X_train, y_train)
#print(g.population[0].head())
#print(g.features.head())
g.fitness(g.population[0])
#[print(type(i)) for i in g.features.iloc[375]]

<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>


In [52]:
random_chromosome = lambda feature_set: [random.random() for f in features]
[random_chromosome(feature_set) for feature_set in X_train]
X_train.columns

Index(['age', 'bmi', 'children', 'sex0', 'sex1', 'smoker0', 'smoker1',
       'region0', 'region1', 'region2', 'region3'],
      dtype='object')

In [53]:
def predict(features, weights):
    return np.dot(features, weights)

In [54]:
[print(i) for i in range(10)]

0
1
2
3
4
5
6
7
8
9


[None, None, None, None, None, None, None, None, None, None]